In [93]:
import httplib
import json
from datetime import datetime, date
import pandas as pd
import time

cities = ['Genova','Torino','Bologna','Firenze','Ancona','Livorno','Roma','Pescara','Bari','Napoli','Taranto','Lecce', 
          'Cosenza', 'Reggio Calabria','Varese','Bergamo','Nizza','Marsiglia','Grenoble','Ginevra','Losanna','Zurigo','Venezia',
         'Trento','Lugano','Brescia','Lione','Padova','Udine','Bolzano','Trieste','Reggio Emilia','Berna','Monaco di Baviera',
          'Verona','Rimini', 'Ravenna','Terni','Salerno','Crotone','Savona','Cannes','Pisa','Como','Pesaro','Treviso','Fiume',
         'Innsbruck']

print(len(cities))

48


In [83]:
conn = httplib.HTTPSConnection("public-api.blablacar.com")

headers = {
        'accept': "application/json",
        'key': "e359fe6292be4078ba1dcc963577813f"
    }

stringa = '/api/v2/trips?fn=Milano&tn=Ancona&locale=it_IT&_format=json&db=2016-12-31&de=2016-12-31&page=1&limit=100&radius=5'

trips = conn.request("GET",stringa, headers=headers)

res = conn.getresponse()
data = res.read().decode("utf-8")

In [84]:
prova = json.loads(data)

prova['pager']['pages']

1

In [86]:
s1 = "/api/v2/trips?fn=Milano&tn=" # destination
s2 = "&locale=it_IT&_format=json&db=" # date begin
s3 = "&de=" # date end
s4 = "&page=" # page num
s5 = "&limit=100&radius=5"    
date_query = '2016-12-31'
c = 'Ancona'

request_url = s1 + c + s2 + date_query + s3 + date_query + s4 + str(1) + s5
print request_url
trips = conn.request("GET",request_url, headers=headers)

res = conn.getresponse()
data = res.read().decode("utf-8")

prova = json.loads(data)


/api/v2/trips?fn=Milano&tn=Ancona&locale=it_IT&_format=json&db=2016-12-31&de=2016-12-31&page=1&limit=100&radius=5


In [113]:
def extract_data(trip):
    depdate = datetime.strptime(trip['departure_date'], "%d/%m/%Y %H:%M:%S")
    depplace = trip['departure_place']
    arrplace = trip['arrival_place']
    
    link = trip['links']['_front']    # link to the page of the trip (available only for trips "in the future")
    
    dep_day = depdate.date()       # date of the trip
    dep_time = depdate.time()      # hour of departure
    dep_city = depplace['city_name']  # city of dep
    dep_lat = depplace['latitude']    # latitude of dep place
    dep_long = depplace['longitude']  # longitude of dep place
    dep_addr = depplace['address']    # address of dep place
    arr_city = arrplace['city_name']  
    arr_lat = arrplace['latitude']
    arr_long = arrplace['longitude']
    arr_addr = arrplace['address']
    distance = int(trip['distance']['value'])
    price = trip['price_with_commission']['value'] # price with commission
    seatsleft = trip['seats_left']                 # seats left 
    seats = trip['seats']                          # seats made available by the driver
    trip_id = trip['permanent_id']                 # trip id
    stops = trip['locations_to_display']           # locations in which the driver will stop
    bookmode = trip['booking_mode']                # could be MANUAL if the driver has to accept your request, AUTO otherwise
    booktype = trip['booking_type']               # payment method: ONLINE, ON BOARD
    
    return (link, dep_day, dep_time, dep_city, dep_lat, dep_long, dep_addr, arr_city, arr_lat, arr_long, arr_addr, distance, price, 
         seatsleft, seats, trip_id, stops, bookmode, booktype)

In [112]:
def tuplelist_to_df(L):
    labels = ['link', 'dep_day', 'dep_time', 'dep_city', 'dep_lat', 'dep_long', 'dep_addr', 'arr_city', 'arr_lat', 'arr_long', 
              'arr_addr', 'distance','price', 'seatsleft', 'seats', 'trip_id', 'stops', 'bookmode', 'booktype']
    df = pd.DataFrame.from_records(L, columns = labels)
    return df

In [118]:
def dict_to_df(d):
    return pd.DataFrame(d.values(), index=pd.MultiIndex.from_tuples(d.keys())).unstack(1)

In [123]:
def get_data(year,month,day_begin,day_end, cities):
    conn = httplib.HTTPSConnection("public-api.blablacar.com")
    
    headers = {
        'accept': "application/json",
        'key': "e359fe6292be4078ba1dcc963577813f"
    }
    
    count = 0
    
    yearmonth = str(year)+'-'+str(month).zfill(2)+'-'
    
    my_data = []
    d_full = {}
    
    s1 = "/api/v2/trips?fn=Milano&tn=" # destination
    s2 = "&locale=it_IT&_format=json&db=" # date begin
    s3 = "&de=" # date end
    s4 = "&page=" # page num
    s5 = "&limit=100&radius=5"    
    for day in xrange(day_begin, day_end+1):
        date_query = yearmonth + str(day).zfill(2)
        print date_query
        for c in cities:
            count += 1
            #print c, count
            request_url = s1 + c + s2 + date_query + s3 + date_query + s4 + str(1) + s5
            
            if count == 100:
                time.sleep(30)
                count = 0
                
            conn.request("GET", request_url, headers=headers) 
            res = conn.getresponse()
            time.sleep(2)
            
            data = res.read().decode("utf-8")
            trips_data = json.loads(data)
            
            if trips_data:
                num_pages = int(trips_data['pager']['pages'])
            
                num_full_trips = int(trips_data['full_trips_count'])
                d_full[(c,date_query)] = num_full_trips    # key is the tuple (city, date)

                for t in trips_data['trips']:
                    my_data.append(extract_data(t))
            
                for p in xrange(2,num_pages+1):
                    request_url = s1 + c + s2 + date_query + s3 + date_query + s4 + str(p) + s5
                    conn.request("GET", request_url, headers=headers)
                    res = conn.getresponse()
                    time.sleep(2)
                    
                    data = res.read().decode("utf-8")
                    trips_data = json.loads(data)                
                    for t in trips_data['trips']:
                        my_data.append(extract_data(t))
                                     
    return tuplelist_to_df(my_data), dict_to_df(d_full)

In [120]:
#December
df_dec, full_dec = get_data(2016,12,30,31,cities)

print len(df_dec), len(full_dec)

2016-12-30
Genova 1
Torino 2
Bologna 3
Firenze 4
Ancona 5
Livorno 6
Roma 7
Pescara 8
Bari 9
Napoli 10
Taranto 11
Lecce 12
Cosenza 13
Reggio Calabria 14
Varese 15
Bergamo 16
Nizza 17
Marsiglia 18
Grenoble 19
Ginevra 20
Losanna 21
Zurigo 22
Venezia 23
Trento 24
Lugano 25
Brescia 26
Lione 27
Padova 28
Udine 29
Bolzano 30
Trieste 31
Reggio Emilia 32
Berna 33
Monaco di Baviera 34
Verona 35
Rimini 36
Ravenna 37
Terni 38
Salerno 39
Crotone 40
Savona 41
Cannes 42
Pisa 43
Como 44
Pesaro 45
Treviso 46
Fiume 47
Innsbruck 48
2016-12-31
Genova 49
Torino 50
Bologna 51
Firenze 52
Ancona 53
Livorno 54
Roma 55
Pescara 56
Bari 57
Napoli 58
Taranto 59
Lecce 60
Cosenza 61
Reggio Calabria 62
Varese 63
Bergamo 64
Nizza 65
Marsiglia 66
Grenoble 67
Ginevra 68
Losanna 69
Zurigo 70
Venezia 71
Trento 72
Lugano 73
Brescia 74
Lione 75
Padova 76
Udine 77
Bolzano 78
Trieste 79
Reggio Emilia 80
Berna 81
Monaco di Baviera 82
Verona 83
Rimini 84
Ravenna 85
Terni 86
Salerno 87
Crotone 88
Savona 89
Cannes 90
Pisa 91
Como

In [121]:
df_dec.to_csv('./Data/Past/Dec2016.csv', encoding = 'utf-8')
full_dec.to_csv('./Data/Past/fullDec2016.csv')

In [124]:
df_jan, full_jan = get_data(2017,1,1,31,cities)

print len(df_jan)

2017-01-01
2017-01-02
2017-01-03
2017-01-04
2017-01-05
2017-01-06
2017-01-07
2017-01-08
2017-01-09
2017-01-10
2017-01-11
2017-01-12
2017-01-13
2017-01-14
2017-01-15
2017-01-16
2017-01-17


KeyError: 'pager'